In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 6.5 MB/s 
     |████████████████████████████████| 163 kB 62.5 MB/s 
     |████████████████████████████████| 7.6 MB 32.1 MB/s 


In [2]:
from transformers import pipeline
import pandas as pd
import heapq
import numpy as np

In [3]:
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [4]:
df_skills = pd.read_csv('/content/drive/MyDrive/Omdena/AI for soft skills/skills_df.csv')

In [5]:
df_skills

,Skills category,Skills sub-category,Finer-aspects,N
0,Communication collaboration and emotional inte...,Collaboration,Active listening,1
1,Communication collaboration and emotional inte...,Collaboration,Active participation in activities,1
2,Communication collaboration and emotional inte...,Collaboration,Inclusiveness of thought and action,1
3,Communication collaboration and emotional inte...,Collaboration,Active verbal skills,1
4,Communication collaboration and emotional inte...,Collaboration,Adaptability,1
...,...,...,...,...
203,Persistence Initiation thinking Self-esteem Co...,Self-motivation,Goal Setting,1
204,Persistence Initiation thinking Self-esteem Co...,Self-motivation,Time Management,1
205,Persistence Initiation thinking Self-esteem Co...,Self-motivation,Self congratulatory,1
206,Communication collaboration and emotional inte...,Comfort with Ambiguity,Finding meaning in vaguness,1


In [6]:
df_data = pd.read_excel('/content/drive/MyDrive/Omdena/AI for soft skills/Finer_aspect_v2.xlsx')
df_data

,Skills category,Skills sub-category,Finer aspects,Content
0,Communication collaboration and emotional inte...,Collaboration,Adaptability,Adaptability is a soft skill that means you ea...
1,Communication collaboration and emotional inte...,Collaboration,Adaptability,If you’re someone who loves processes and orde...
2,Communication collaboration and emotional inte...,Collaboration,Adaptability,A growth mindset is a vital part of adaptabili...
3,Communication collaboration and emotional inte...,Collaboration,Adaptability,"Change isn’t steady or predictable for anyone,..."
4,Communication collaboration and emotional inte...,Collaboration,Adaptability,"As the cliché goes, change is the one constant..."
...,...,...,...,...
187,Management planning organization Startegic vis...,Leadership,Influencing Skill,Persuasion involves using facts and arguments ...
188,Management planning organization Startegic vis...,Leadership,Influencing Skill,Build trust. You are in a better position to i...
189,Management planning organization Startegic vis...,Leadership,Influencing Skill,Focus on actions. It is important to prove you...
190,Management planning organization Startegic vis...,Leadership,Influencing Skill,Deliver reliable results. Consistency in perfo...


In [ ]:
df_data['Skills category'].value_counts()

Communication collaboration and emotional intelligence                    125
Creativity Critical thinking and positive thinking                         57
Management planning organization Startegic vision and Entrepruenership     10
Name: Skills category, dtype: int64

In [7]:
finer_aspects_set1 = df_data[df_data['Skills category'] == 'Communication collaboration and emotional intelligence']['Finer aspects'].unique()
finer_aspects_set1

array(['Adaptability', 'Co-ordination skills', 'Taking responsibility',
       'Team Player', "Connecting situations' dots together",
       'Expressing opinion', 'Persuasion skills', 'Tone of expression',
       'Decision making based on both emotions and rational thinking',
       'Prioritizing actions over emotions', 'Anticipating outcomes',
       'Foreseeing situations'], dtype=object)

In [8]:
df_data_set1 = df_data[df_data['Skills category'] == 'Communication collaboration and emotional intelligence']
df_data_set1

,Skills category,Skills sub-category,Finer aspects,Content
0,Communication collaboration and emotional inte...,Collaboration,Adaptability,Adaptability is a soft skill that means you ea...
1,Communication collaboration and emotional inte...,Collaboration,Adaptability,If you’re someone who loves processes and orde...
2,Communication collaboration and emotional inte...,Collaboration,Adaptability,A growth mindset is a vital part of adaptabili...
3,Communication collaboration and emotional inte...,Collaboration,Adaptability,"Change isn’t steady or predictable for anyone,..."
4,Communication collaboration and emotional inte...,Collaboration,Adaptability,"As the cliché goes, change is the one constant..."
...,...,...,...,...
120,Communication collaboration and emotional inte...,Communication collaboration and emotional inte...,Foreseeing situations,"As a leader, you're already seeking out news a..."
121,Communication collaboration and emotional inte...,Communication collaboration and emotional inte...,Foreseeing situations,The natural next step of scanning is to use yo...
122,Communication collaboration and emotional inte...,Communication collaboration and emotional inte...,Foreseeing situations,Foresight is not about looking into a crystal ...
123,Communication collaboration and emotional inte...,Communication collaboration and emotional inte...,Foreseeing situations,"In other words, although foresight is looking ..."


**Multi-Label Zero SHot classification**

In [9]:
def multi_label_ZSC(dataframe, finer_aspects_set1):
  
  mlzsc = []

  for i in range(len(dataframe)):
    sequence_to_classify = dataframe['Content'][i]
    candidate_labels = finer_aspects_set1
    result = classifier(sequence_to_classify, candidate_labels)
    labels = np.array(result['labels'])
    scores = np.array(result['scores'])

    scores_threshold =  scores[scores > 0.1]
    multi_labels = ''
    for j in range(len(scores_threshold)):
      if j == len(scores_threshold) - 1:
        multi_labels = multi_labels + labels[j]
      else:
        multi_labels = multi_labels + labels[j] + ', '

    mlzsc.append(multi_labels)
  return mlzsc

   
    


   


    


In [10]:
df_trail = df_data_set1[0:10]


In [11]:
labels = multi_label_ZSC(df_trail, finer_aspects_set1)

In [12]:
df_trail['MLZSC'] = labels
df_trail

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Skills category,Skills sub-category,Finer aspects,Content,MLZSC
0,Communication collaboration and emotional inte...,Collaboration,Adaptability,Adaptability is a soft skill that means you ea...,Adaptability
1,Communication collaboration and emotional inte...,Collaboration,Adaptability,If you’re someone who loves processes and orde...,"Adaptability, Tone of expression, Expressing o..."
2,Communication collaboration and emotional inte...,Collaboration,Adaptability,A growth mindset is a vital part of adaptabili...,Adaptability
3,Communication collaboration and emotional inte...,Collaboration,Adaptability,"Change isn’t steady or predictable for anyone,...","Adaptability, Tone of expression"
4,Communication collaboration and emotional inte...,Collaboration,Adaptability,"As the cliché goes, change is the one constant...","Adaptability, Connecting situations' dots toge..."
5,Communication collaboration and emotional inte...,Collaboration,Adaptability,One key method that can help you develop your ...,"Adaptability, Connecting situations' dots toge..."
6,Communication collaboration and emotional inte...,Collaboration,Adaptability,Being adaptable also means being willing to le...,Adaptability
7,Communication collaboration and emotional inte...,Collaboration,Adaptability,Another method that can help you develop your ...,"Adaptability, Tone of expression"
8,Communication collaboration and emotional inte...,Collaboration,Adaptability,"As you develop throughout your career, you mig...","Expressing opinion, Anticipating outcomes, Ton..."
9,Communication collaboration and emotional inte...,Collaboration,Adaptability,Learning to acknowledge changes in your career...,"Adaptability, Foreseeing situations, Connectin..."


**Content Aggregation**

In [42]:
def aggreagation(dataframe):
  new_data = {}

  for i in range(len(dataframe)):
    curr_labels = dataframe['MLZSC'][i].split(', ')

    for label in curr_labels:
      if label not in new_data:
        new_data[label] = []
      new_data[label].append(dataframe['Content'][i])
  
  agg = []
  finer_aspect = []
  for k in new_data:
    paras = new_data[k]
    for l in range(len(paras)):
      agg.append(paras[l])
      finer_aspect.append(k)

  new_df = pd.DataFrame({'Finer aspects': finer_aspect, 'Aggregated content': agg})
  return new_df




In [43]:
new_data = aggreagation(df_trail)
new_data

,Finer aspects,Aggregated content
0,Adaptability,Adaptability is a soft skill that means you ea...
1,Adaptability,If you’re someone who loves processes and orde...
2,Adaptability,A growth mindset is a vital part of adaptabili...
3,Adaptability,"Change isn’t steady or predictable for anyone,..."
4,Adaptability,"As the cliché goes, change is the one constant..."
5,Adaptability,One key method that can help you develop your ...
6,Adaptability,Being adaptable also means being willing to le...
7,Adaptability,Another method that can help you develop your ...
8,Adaptability,Learning to acknowledge changes in your career...
9,Tone of expression,If you’re someone who loves processes and orde...


In [28]:
new_data.keys()

dict_keys(['Adaptability', 'Tone of expression', 'Expressing opinion', "Connecting situations' dots together", 'Foreseeing situations', 'Anticipating outcomes', 'Taking responsibility'])

In [39]:
len(new_data['Adaptability'])

9

In [40]:
agg = []
finer_aspect = []
for i in new_data:
  paras = new_data[i]
  for j in range(len(paras)):
    agg.append(paras[j])
    finer_aspect.append(i)

  